In [1]:
using Pkg
Pkg.instantiate()

In [10]:
using Revise, CairoMakie
using ColonyImages
using DataFrames, CSV, FileIO, Dates, Serialization
import ColorSchemes as co 

In [11]:
df = DataFrame(data_set =String[], colony = String[], time = Int[], 
    metric_OG =Vector{Vector{Int64}}(undef,0), metric_cov = Vector{Vector{Int64}}(undef,0),
    pair_OG =Vector{Vector{Int64}}(undef,0),pair_cov =Vector{Vector{Int64}}(undef,0),OG_size = Int[])

Row,data_set,colony,time,metric_OG,metric_cov,pair_OG,pair_cov,OG_size
,String,String,Int64,Array…,Array…,Array…,Array…,Int64


In [ ]:
para = parameters(
        im_size = [6000,6000],
        time_points = [0:8.0:180...],
        colony_nr = 2);

#simulations = ["Random", "Finger_weak", "Finger_strong"]
simulations = ["Random"]
plot_sim = false

In [ ]:
name_of_sim = "$(para.colony_nr)_col_$(length(simulations))_long_growth_random_$(para.im_size[1])"
fig_path = joinpath("data", "artificial",name_of_sim)
path_metric = joinpath("data", "metrics")
mkpath(path_metric)
mkpath(fig_path)

## Create empty datastructure for colony images

In [4]:
pixel_to_add_vec = Int[];
vec_of_sims = Vector{Vector{BitArray{3}}}(undef, 0)
for i in 1:length(simulations)
    img_int_vec_1 = Vector{BitArray{3}}(undef, 0)
    for i in 1:para.colony_nr
        hans =BitArray(zeros(Bool, para.im_size...,para.stacks))
        push!(img_int_vec_1, hans)
    end
    push!(vec_of_sims, img_int_vec_1)
end

UndefVarError: UndefVarError: `simulations` not defined

In [5]:
pixel_to_add_vec = Int[];
for (x,sim) in enumerate(simulations)
    for (i,colony) in enumerate(vec_of_sims[x])
        for (j,t) in enumerate(para.time_points)
            # use view to work on array directly 
            int_img  = @view colony[:,:,j]
            dir_vec = generate_dir_vec(para)
            if j == 1
                build_artifical_colony!(para.Center,int_img,para.radius_colony,para.Points)
                pixel_to_add_vec = para.pixel_to_add(int_img)
            else
                # for unkown reason views require [:] to make assignments 
                int_img[:] = colony[:,:,j-1]
                if sim == "Random"
                    expand_colony_radom_cov!(int_img,pixel_to_add_vec[j-1])
                elseif sim == "Finger_weak"
                    expand_colony_finger_radom_cov!(int_img,pixel_to_add_vec[j-1],dir_vec)
                elseif sim == "Finger_strong"
                    expand_colony_finger_radom_cov!(int_img,pixel_to_add_vec[j-1],dir_vec, still_spawn_rate= 0.0, dir_match_rate = 0.9995)
                end
            end
        end
    end   
end

UndefVarError: UndefVarError: `simulations` not defined

### Plot first colony test wise to see if everything worked as intended

In [6]:
fig_test = plot_time_series_cov_centroid(vec_of_sims[1][1:1], para)

BoundsError: BoundsError: attempt to access 0-element Vector{Vector{BitArray{3}}} at index [1]

###  Either plot sim into artifical data folder or not to save computation time

In [7]:
if plot_sim 
    for (i,name) in enumerate(simulations)
        fig, data_set_name = plot_and_save_time_series_metrics!(vec_of_sims[i], para,df, name_data = name)
        save(joinpath(fig_path, data_set_name)*".png", fig)
    end
else
    for (i,name) in enumerate(simulations)
        data_set_name = save_time_series_metrics!(vec_of_sims[i], para,df, name_data = name)
    end
end

UndefVarError: UndefVarError: `plot_sim` not defined

In [ ]:
first(df,2)

In [19]:
serialize(joinpath(path_metric,"$(name_of_sim*Dates.format(now(), "yyyy_mm_dd")).jls"),df)
CSV.write(joinpath(path_metric,"$(name_of_sim*Dates.format(now(), "yyyy_mm_dd")).csv"),df)

"data\\metrics\\2_col_1_long_growth_random2024_03_11.csv"